In [ ]:
! pip install gmaps

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import datetime
from config import git_key
from config import api_key
import gmaps
# gmaps.configure(api_key="api_key")
%matplotlib notebook

In [ ]:
gmaps.configure(api_key="YOUR GOOGLE MAPS API_KEY HERE")

In [ ]:
def corona_df(git_key, branch):
    from github import Github
    import requests
    import io
    # First create a Github instance  using username and password
    # g = Github("user", "password")
    # # or using an access token
    g = Github(git_key)
    repo = g.get_repo("CSSEGISandData/COVID-19")
    # URL for GET requests to retrieve coronavirus data
    contents = repo.get_contents(branch)
    df = pd.read_csv(io.StringIO(contents.decoded_content.decode('utf-8')))
    return df

In [ ]:
# Pull "APIs" using defined formula above
confirmed_df = corona_df(git_key,"/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv")
deaths_df = corona_df(git_key,"/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv")
recovered_df = corona_df(git_key,"/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv")

In [ ]:
confirmed_df.head()

In [ ]:
def clean_df(df):
    
    df = df.fillna(value=0)                                          # Fill NaN with zero values
    df = df.sort_values(by=df.columns[-1], ascending=False)          # Sort by highest value of most recent recorded date

    return df

In [ ]:
df_confirmed_clean = clean_df(confirmed_df)
df_confirmed_clean

In [ ]:
locations = df_confirmed_clean[["Lat", "Long"]]

In [ ]:
recent = df_confirmed_clean.iloc[:][df_confirmed_clean.columns[-1]]

In [ ]:
# Define fig saving function
def fig_save(fig, file_name):
    import os
    path = os.path.join("images", file_name)
    fig.savefig(path)

In [ ]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=recent,
                                dissipating=False, max_intensity=10,
                                point_radius=1)
fig.add_layer(heat_layer)

fig